The main goal of the analysis will focus on prediction of life expectancy of people depending on various features and interpreting which model is the best for predicting the output. The main objective of this analysis is to find the best regression model for predicting life expectancy and to find whether there is a linear relationship with the features and life expectancy. The data I have chosen includes the life expectencies of people around the world from different countries from 2000 to 2015 and also many features such as immunization, mortality, economic, social, population and many other feature with a total of 22 columns.

Since Country and Year was not directly related in terms of predicting life expectancy I dropped those columns by using pandas library. The status columns was also categorical, so I replaced 'Developed' with 1 and 'Developing' as 0 to prepare it for regression. Moreover, there were also many missing values especially in the columns 'Population' and 'Hepatitis B' with more than 500 missing values. Therefore, I dropped all the rows that had a missing value in these columns. For the missing values in all the other columns, I filled the missing values with the mean of the relevant column. Finally, I used Standard Scaler to scale the features for regression.

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, 
                                   PolynomialFeatures)
from scipy.special import inv_boxcox

df = pd.read_csv('Life_Expectancy_Data.csv', sep=',')
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [45]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

In [46]:
df.describe()


,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


In [47]:
df.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [48]:
df.columns = df.columns.str.replace(' ', '')

In [49]:
df = df.dropna(subset=["Population", 'HepatitisB'])

In [50]:
df = df.drop(['Country', 'Year'], axis=1)

In [51]:
for i in range(0, len(df.columns)):
    if df[df.columns[i]].isnull().values.any() and df.columns[i] != 'Population' and df.columns[i] != 'HepatitisB':
        mean = df[df.columns[i]].mean()
        df[df.columns[i]].fillna(mean, inplace = True)
        
df['Status'] = df['Status'].replace(['Developing', 'Developed'], [0, 1])
        

I used 3 different linear regression methods which were basic linear regression, lasso regression and ridge regression. I used polynomial regression in all of these regressions with a degree of 2. My test size was 30% of the data and I used train test split method according to that. All of the regressions used the same train test split which made it suitable for comparing the accuracy of these models by using the same inputs.

In Lasso regression I used an alpha value of 0.001, because that was the alpha values that gave me the highest result of the R^2 ammong the values that I tested and I used an alpha value of 1 for Ridge regression for the same reason. 

In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
y_col = "Lifeexpectancy"

X = df.drop(y_col, axis=1)
y = df[y_col]

pf = PolynomialFeatures(degree=2, include_bias=False)
X_pf = pf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=34840)

s = StandardScaler()
X_train_s = s.fit_transform(X_train)


X_test_s = s.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_s,y_train)
lr_pred = lr.predict(X_test_s)

print(r2_score(lr_pred,y_test))
print(np.sqrt(mean_squared_error(lr_pred, y_test)))

0.7279304867549585
4.462418561851701


In [56]:
from sklearn.linear_model import Lasso
las = Lasso(alpha = 0.001, max_iter=100000)
X_train_s = s.fit_transform(X_train)
las.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_las = las.predict(X_test_s)
print(r2_score(y_pred_las, y_test))
print(np.sqrt(mean_squared_error(y_pred_las, y_test)))

0.8861990833642179
2.742210352798408


In [57]:
from sklearn.linear_model import Ridge

r = Ridge(alpha = 1)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_ridge = r.predict(X_test_s)
print(r2_score(y_pred_ridge, y_test))
print(np.sqrt(mean_squared_error(y_pred_ridge, y_test)))

0.8835063127192041
2.762958932708856


To interpret the accuracy of the models I found the R^2 and RMSE of each model which were the following:
Basic Linear Regression R^2: 0.7279304867549585, RMSE: 4.462418561851701
Lasso Regression R^2: 0.8861990833642179, RMSE: 2.742210352798408
Ridge Regression: R^2: 0.8835063127192041, RMSE: 2.762958932708856

As a result of these to most successful model in terms of accuracy was the one that used Lasso regression since it had the highest R^2 and the lowest RMSE. The ridge regression is close to the results of Lasso regression. However, we can obviously notice that regularization is crucially important for improving the prediction accuracy of our models. Furthermore, since R^2 was close to one, we can conlude that there is a linear relationship since the features helped us successfully predict the output which was one of the main objectives of the analysis.

To further improve our models, I could use logarithmic or boxcox transformation for life expectancy since having a normal distribution is essential for regression. This would probably improve the accuracy of basic linear regression. Moreover, we can probably achieve higher accuracies by testing different alpha values in regularization and by using cross validation. Finally in order to improve the complexity of the project the features could be explored more deeply such as by finding the coefficients of the features which would help us learn the features that have the most effect on life expectancy. This would definitely provide valuable information for everyone.

Source of the dataset:
Kumar Rajarshi. (2018, February 10). Life expectancy (WHO). Kaggle. Retrieved May 30, 2022, from https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who 